In [14]:
#!/usr/bin/env python
# coding: utf-8
INPUT_CSV = 'nodelist.csv'


import ldms_ishaan_backend as ldms_backend
get_ipython().run_line_magic('matplotlib', 'notebook')
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import widgets
import tokenize
#from matplotlib.widgets import Button
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
%matplotlib notebook
%matplotlib inline

import time
import numpy as np
import IPython

#Default Values
global aries_rtr_id
global r
global c

#Router tile variables
aries_rtr_id = 'c4-0c2s8'
r = 0
c = 0
time_temp =0

#NIC Variables
nid_inp = 'nodeID'

#Latency variables
nid1_inp=0
nid2_inp=0


#Variable Update
def router_clicked(b):
    global aries_rtr_id
    global r
    global c
    global time_temp
    aries_rtr_id = rtr_id_inp.value
    r = int(r_inp.value)
    c = int(c_inp.value)
    time_temp= int(c_time_temp.value)
    drawit(1,"Router: ", aries_rtr_id)
    
def nic_clicked(b):
    global node_id
    global time_temp
    node_id = nid_inp.value
    time_temp= int(nic_time_temp.value)
    drawit(2,"Node :",node_id)
    
def latency_clicked(b):
    global n1
    global n2
    nid1 = nid1_inp.value
    nid2=nid2_inp.value
    drawit(3,nid1,nid2)
    
def app_clicked(b):
    drawapp()
    
    
#Title
html_title = widgets.HTML(value = f"<c><b><font color='DarkBlue' size = \"+1\"> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;LDMS Live</b> </c>")
                          
                          
#Router Counter Tab
rtr_id_inp = widgets.Text(placeholder='Aries Router ID',layout=widgets.Layout(width='150px', height='10px'))
r_inp = widgets.Text( placeholder = 'Row',layout=widgets.Layout(width='50px', height='10px'))
c_inp = widgets.Text(placeholder = 'Col',layout=widgets.Layout(width='50px', height='10px'))
c_time_temp = widgets.Text(placeholder = 'DeltaTime',layout=widgets.Layout(width='100px', height='10px'))
router_button = widgets.Button(description='Add Graphs',layout=widgets.Layout(width='150px', height='28px'))
router_button.on_click(router_clicked)
hbox1 = widgets.HBox([rtr_id_inp,r_inp,c_inp,c_time_temp,router_button])

#NIC Counter Tab
nid_inp = widgets.Text(placeholder='Node ID',layout=widgets.Layout(width='150px', height='10px'))
nic_time_temp = widgets.Text(placeholder = 'DeltaTime',layout=widgets.Layout(width='100px', height='10px'))
nic_button = widgets.Button(description='Add Graphs',layout=widgets.Layout(width='150px', height='28px'))
nic_button.on_click(nic_clicked)
hbox2 = widgets.HBox([nid_inp,nic_time_temp,nic_button])


#App Placement Tab
app_button = widgets.Button(description='Show Apps',layout=widgets.Layout(width='150px', height='28px'))
app_button.on_click(app_clicked)
hbox3 = widgets.HBox([app_button])


#Latency Estimate
nid1_inp = widgets.Text(placeholder = 'NID1',layout=widgets.Layout(width='50px', height='10px'))
nid2_inp = widgets.Text(placeholder = 'NID2',layout=widgets.Layout(width='50px', height='10px'))
latency_button = widgets.Button(description='Add Graphs',layout=widgets.Layout(width='150px', height='28px'))
latency_button.on_click(latency_clicked)
hbox4 = widgets.HBox([nid1_inp,nid2_inp,latency_button])

tab_contents = ['Router Counters','NIC Counters','App Placement','Latency Estimate']
children = [hbox1,hbox2,hbox3,hbox4]
tab = widgets.Tab()
tab.children = children
for i in range(len(children)):
    tab.set_title(i, tab_contents[i])


#Display static fields
display(html_title)
display(tab)

output = widgets.Output()
display(output)


#Graph Update
def drawit(graph,tab,comp):
     with output:
        clear_output()
        sns.set()
        sns.set(style='whitegrid')
        fig = plt.figure(figsize=(20,5))
        plt.clf()
        fig.clf()
        ax = fig.subplots(1,2)
        fig.tight_layout(pad = 5.0)
        ax[0].set_ylabel('Stalls/sec')
        ax[0].set_xlabel('Time (sec)')
        ax[0].set_title('Congestion')
        ax[1].set_ylabel('Flits/sec')
        ax[1].set_xlabel('Time (sec)')
        ax[1].set_title('Traffic')
        if graph is 1:
            df_out = ldms_backend.get_processed_df(aries_rtr_id,r,c,100-time_temp,80-time_temp)
        if graph is 2:
            df_out = ldms_backend.get_processed_df_nic2proc(aries_rtr_id,r,c,100-time_temp,80-time_temp)
        sns.lineplot(x='time',y='stall',data=df_out,ax=ax[0])
        sns.lineplot(x='time',y='flit',data=df_out,ax=ax[1]);
        fig.suptitle(tab + " "+ comp)
        fig.canvas.draw()
        show_inline_matplotlib_plots()

#Read the list of nodes from file
def getdataframe():

    rows=364
    cols=6
    matrix = [[0] * cols for i in range(rows)]
    #print (matrix)
    df = pd.DataFrame.from_records(matrix)
    arrtoknum={}
    arrtokval={}
    i=0
    with tokenize.open(INPUT_CSV) as f:
        tokens = tokenize.generate_tokens(f.readline)
        for toknum, tokval, _, _, _  in tokens:
         #   print(i," ",toknum," ",tokval)
        #for token in tokens:
            arrtoknum.update({i:toknum})
            arrtokval.update({i:tokval})
            i=i+1

    totaltoks= len(arrtoknum)

    i=0
    numbers=[]
    while (i<=totaltoks) :
        toknumx= arrtoknum.get(i)
        tokvalx=arrtokval.get(i)

        if toknumx==2 and int(tokvalx)!=0:
            numbers.append(str(tokvalx))
            #print(toknumx, " ", tokvalx)
        if tokvalx=="-":
            k=1
            prevval=int(arrtokval.get(i-1))
            nextval=int(arrtokval.get(i+1))
            if nextval==0:
                i=i+1
                nextval=int(arrtokval.get(i+1))
            ncount= int(nextval) - int(prevval)
            #print("i ", i, "diff ", ncount, " prev ",prevval, " next ", nextval )
            while(k <= ncount) :
                tokvalx=str(prevval+k)
                numbers.append(tokvalx)
                #print(toknumx, " ", tokvalx)
                k=k+1
            i=i+1
        i=i+1
    #print(numbers)

    for j in numbers:
        ninp=int(j)
        if ninp > 5:
            x=divmod(ninp,6)
            #print("Row=", x[0]," Col=", x[1])
            df[x[1]][x[0]]=1
    return df

  
#Draw App layout
def drawapp():
    with output:
        clear_output()
        sns.set()
        sns.set(style='whitegrid')
        fig = plt.figure(figsize=(10,30))
        plt.clf()
        fig.clf()
        data=getdataframe()
        #data = np.random.rand(64, 6)
           
        heat_map = sns.heatmap(data,cmap="YlGnBu")
        fig.canvas.draw()
        show_inline_matplotlib_plots()
        #plt.show()



HTML(value='<c><b><font color=\'DarkBlue\' size = "+1"> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;…

Output()